In [1]:
!pip install -q torch torchvision datasets timm transformers


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


# DINOv2

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from timm.models.layers import to_2tuple
from tqdm import tqdm
import numpy as np


class DINOv2Classifier(nn.Module):
    def __init__(self, img_size=32, patch_size=8, num_classes=10):
        super(DINOv2Classifier, self).__init__()
        self.dino = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14', pretrained=False)
        self.dino.patch_embed.img_size = to_2tuple(img_size)
        self.dino.patch_embed.patch_size = to_2tuple(patch_size)
        self.dino.head = nn.Linear(self.dino.embed_dim, num_classes)

    def forward(self, x):
        return self.dino(x)


def create_imbalanced_dataset(dataset, imb_type='step', imb_factor=0.01):
    cls_num = 10
    img_max = len(dataset.data) // cls_num
    img_num_per_cls = []

    if imb_type == 'exp':
        for cls_idx in range(cls_num):
            num = img_max * (imb_factor ** (cls_idx / (cls_num - 1.0)))
            img_num_per_cls.append(int(num))
    elif imb_type == 'step':
        for cls_idx in range(cls_num // 2):
            img_num_per_cls.append(img_max)
        for cls_idx in range(cls_num // 2):
            img_num_per_cls.append(int(img_max * imb_factor))
    else:
        img_num_per_cls.extend([img_max] * cls_num)

    new_data = []
    new_targets = []
    targets_np = np.array(dataset.targets)

    for cls_idx, img_num in enumerate(img_num_per_cls):
        cls_indices = np.where(targets_np == cls_idx)[0]
        selected_indices = np.random.choice(cls_indices, img_num, replace=False)
        new_data.append(dataset.data[selected_indices])
        new_targets.extend([cls_idx] * img_num)

    dataset.data = np.concatenate(new_data, axis=0)
    dataset.targets = new_targets
    return dataset


def train_model(model, train_loader, test_loader, criterion, optimizer, device, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        train_accuracy = 100. * correct / total
        print(f"Epoch {epoch + 1}: Loss = {epoch_loss:.4f}, Accuracy = {train_accuracy:.2f}%")

    test_accuracy = evaluate_model(model, test_loader, device)
    return test_accuracy


def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)

            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100. * correct / total
    return accuracy


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

exp_dataset = create_imbalanced_dataset(datasets.CIFAR10(root='./data', train=True, download=True, transform=transform), 'exp', 0.01)
step_dataset = create_imbalanced_dataset(datasets.CIFAR10(root='./data', train=True, download=True, transform=transform), 'step', 0.01)

batch_size = 128
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

reduction_factors = [0.25, 0.5, 0.75, 1]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

results = {}
for name, dataset in [("Balanced", train_dataset), ("Exp Imbalanced", exp_dataset), ("Step Imbalanced", step_dataset)]:
    results[name] = {}
    for factor in reduction_factors:
        subset_size = int(len(dataset) * factor)
        indices = np.random.choice(len(dataset), subset_size, replace=False)
        subset = Subset(dataset, indices)

        train_loader = DataLoader(subset, batch_size=batch_size, shuffle=True, num_workers=4)

        print(f"\nTraining on {name} dataset with {factor * 100:.0f}% data")
        model = DINOv2Classifier(img_size=32, patch_size=8, num_classes=10).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=1e-4)

        test_accuracy = train_model(model, train_loader, test_loader, criterion, optimizer, device, epochs=5)
        results[name][factor] = test_accuracy

for factor in reduction_factors:
    balanced_acc = results["Balanced"][factor]
    for name in ["Exp Imbalanced", "Step Imbalanced"]:
        acc_diff = balanced_acc - results[name][factor]
        print(f"Accuracy Difference ({name} vs Balanced) at {factor * 100:.0f}% data: {acc_diff:.2f}%")

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Using device: cuda

Training on Balanced dataset with 25% data


Using cache found in /home/mlcore/.cache/torch/hub/facebookresearch_dinov2_main
Epoch 1/5: 100%|██████████| 98/98 [00:02<00:00, 46.20it/s]


Epoch 1: Loss = 2.0140, Accuracy = 28.18%


Epoch 2/5: 100%|██████████| 98/98 [00:02<00:00, 46.17it/s]


Epoch 2: Loss = 1.7502, Accuracy = 35.83%


...

Epoch 4: Loss = 0.8744, Accuracy = 67.35%


Epoch 5/5: 100%|██████████| 198/198 [00:04<00:00, 48.50it/s]


Epoch 5: Loss = 0.7793, Accuracy = 71.18%


Evaluating: 100%|██████████| 79/79 [00:00<00:00, 98.83it/s] 

Accuracy Difference (Exp Imbalanced vs Balanced) at 25% data: 18.32%
Accuracy Difference (Step Imbalanced vs Balanced) at 25% data: 13.37%
Accuracy Difference (Exp Imbalanced vs Balanced) at 50% data: 21.79%
Accuracy Difference (Step Imbalanced vs Balanced) at 50% data: 17.39%
Accuracy Difference (Exp Imbalanced vs Balanced) at 75% data: 22.53%
Accuracy Difference (Step Imbalanced vs Balanced) at 75% data: 18.61%
Accuracy Difference (Exp Imbalanced vs Balanced) at 100% data: 24.57%
Accuracy Difference (Step Imbalanced vs Balanced) at 100% data: 21.09%


In [46]:
import pandas as pd

data = []
for factor in reduction_factors:
    row = {"Reduction Factor": f"{int(factor * 100)}%"}
    for name in ["Exp Imbalanced", "Step Imbalanced", "Balanced"]:
        row[name] = results[name][factor]
    data.append(row)
pd.DataFrame(data)

,Reduction Factor,Exp Imbalanced,Step Imbalanced,Balanced
0,25%,23.52,28.47,41.84
1,50%,25.10,29.50,46.89
2,75%,27.97,31.89,50.50
3,100%,29.00,32.48,53.57


# ViT

In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from timm.models.layers import to_2tuple
from tqdm import tqdm
import numpy as np


class ViTClassifier(nn.Module):
    def __init__(self, img_size=32, num_classes=10):
        super(ViTClassifier, self).__init__()
        self.vit = torch.hub.load('facebookresearch/dino:main', 'dino_vitb8', pretrained=False)
        self.vit.patch_embed.img_size = to_2tuple(img_size)
        self.vit.head = nn.Linear(self.vit.embed_dim, num_classes)

    def forward(self, x):
        return self.vit(x)


def create_imbalanced_dataset(dataset, imb_type='step', imb_factor=0.01):
    cls_num = 10
    img_max = len(dataset.data) // cls_num
    img_num_per_cls = []

    if imb_type == 'exp':
        for cls_idx in range(cls_num):
            num = img_max * (imb_factor ** (cls_idx / (cls_num - 1.0)))
            img_num_per_cls.append(int(num))
    elif imb_type == 'step':
        for cls_idx in range(cls_num // 2):
            img_num_per_cls.append(img_max)
        for cls_idx in range(cls_num // 2):
            img_num_per_cls.append(int(img_max * imb_factor))
    else:
        img_num_per_cls.extend([img_max] * cls_num)

    new_data = []
    new_targets = []
    targets_np = np.array(dataset.targets)

    for cls_idx, img_num in enumerate(img_num_per_cls):
        cls_indices = np.where(targets_np == cls_idx)[0]
        selected_indices = np.random.choice(cls_indices, img_num, replace=False)
        new_data.append(dataset.data[selected_indices])
        new_targets.extend([cls_idx] * img_num)

    dataset.data = np.concatenate(new_data, axis=0)
    dataset.targets = new_targets
    return dataset


def train_model(model, train_loader, test_loader, criterion, optimizer, device, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        train_accuracy = 100. * correct / total
        print(f"Epoch {epoch + 1}: Loss = {epoch_loss:.4f}, Accuracy = {train_accuracy:.2f}%")

    test_accuracy = evaluate_model(model, test_loader, device)
    return test_accuracy


def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)

            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100. * correct / total
    return accuracy


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

exp_dataset = create_imbalanced_dataset(datasets.CIFAR10(root='./data', train=True, download=True, transform=transform), 'exp', 0.01)
step_dataset = create_imbalanced_dataset(datasets.CIFAR10(root='./data', train=True, download=True, transform=transform), 'step', 0.01)

batch_size = 128
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

reduction_factors = [0.25, 0.5, 0.75, 1]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

results = {}
for name, dataset in [("Balanced", train_dataset), ("Exp Imbalanced", exp_dataset), ("Step Imbalanced", step_dataset)]:
    results[name] = {}
    for factor in reduction_factors:
        subset_size = int(len(dataset) * factor)
        indices = np.random.choice(len(dataset), subset_size, replace=False)
        subset = Subset(dataset, indices)

        train_loader = DataLoader(subset, batch_size=batch_size, shuffle=True, num_workers=4)

        print(f"\nTraining on {name} dataset with {factor * 100:.0f}% data")
        model = ViTClassifier(img_size=32, num_classes=10).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=1e-4)

        test_accuracy = train_model(model, train_loader, test_loader, criterion, optimizer, device, epochs=5)
        results[name][factor] = test_accuracy

for factor in reduction_factors:
    balanced_acc = results["Balanced"][factor]
    for name in ["Exp Imbalanced", "Step Imbalanced"]:
        acc_diff = balanced_acc - results[name][factor]
        print(f"Accuracy Difference ({name} vs Balanced) at {factor * 100:.0f}% data: {acc_diff:.2f}%")


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Using device: cuda

Training on Balanced dataset with 25% data


Using cache found in /home/mlcore/.cache/torch/hub/facebookresearch_dino_main
Epoch 1/5: 100%|██████████| 98/98 [00:04<00:00, 23.30it/s]


Epoch 1: Loss = 2.1450, Accuracy = 26.98%


Epoch 2/5: 100%|██████████| 98/98 [00:04<00:00, 23.84it/s]


Epoch 2: Loss = 1.7572, Accuracy = 35.12%


...

Epoch 4: Loss = 0.8338, Accuracy = 69.04%


Epoch 5/5: 100%|██████████| 198/198 [00:07<00:00, 24.77it/s]


Epoch 5: Loss = 0.7387, Accuracy = 72.91%


Evaluating: 100%|██████████| 79/79 [00:01<00:00, 57.76it/s]

Accuracy Difference (Exp Imbalanced vs Balanced) at 25% data: 26.21%
Accuracy Difference (Step Imbalanced vs Balanced) at 25% data: 16.30%
Accuracy Difference (Exp Imbalanced vs Balanced) at 50% data: 27.19%
Accuracy Difference (Step Imbalanced vs Balanced) at 50% data: 21.27%
Accuracy Difference (Exp Imbalanced vs Balanced) at 75% data: 25.58%
Accuracy Difference (Step Imbalanced vs Balanced) at 75% data: 21.49%
Accuracy Difference (Exp Imbalanced vs Balanced) at 100% data: 27.39%
Accuracy Difference (Step Imbalanced vs Balanced) at 100% data: 23.78%


In [50]:
data = []
for factor in reduction_factors:
    row = {"Reduction Factor": f"{int(factor * 100)}%"}
    for name in ["Exp Imbalanced", "Step Imbalanced", "Balanced"]:
        row[name] = results[name][factor]
    data.append(row)
pd.DataFrame(data)

,Reduction Factor,Exp Imbalanced,Step Imbalanced,Balanced
0,25%,18.17,28.08,44.38
1,50%,25.72,31.64,52.91
2,75%,29.27,33.36,54.85
3,100%,30.08,33.69,57.47


# BEiT

In [51]:
import torch
import torch.nn as nn
from torch.hub import load
from transformers import BeitModel, BeitConfig
from timm.models.layers import to_2tuple


class BeitClassifier(nn.Module):
    def __init__(self, img_size=32, patch_size=8, num_classes=10):
        super(BeitClassifier, self).__init__()
        self.config = BeitConfig.from_pretrained("microsoft/beit-base-patch16-224-pt22k")
        self.config.image_size = img_size
        self.config.patch_size = patch_size
        self.beit = BeitModel(self.config)
        self.classifier = nn.Linear(self.config.hidden_size, num_classes)

    def forward(self, x):
        outputs = self.beit(pixel_values=x)
        pooled_output = outputs.last_hidden_state.mean(dim=1)
        logits = self.classifier(pooled_output)
        return logits


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
exp_dataset = create_imbalanced_dataset(datasets.CIFAR10(root='./data', train=True, download=True, transform=transform), 'exp', 0.01)
step_dataset = create_imbalanced_dataset(datasets.CIFAR10(root='./data', train=True, download=True, transform=transform), 'step', 0.01)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

batch_size = 128
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

reduction_factors = [0.25, 0.5, 0.75, 1]
results = {}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

for name, dataset in [("Balanced", train_dataset), ("Exp Imbalanced", exp_dataset), ("Step Imbalanced", step_dataset)]:
    results[name] = {}
    for factor in reduction_factors:
        subset_size = int(len(dataset) * factor)
        indices = np.random.choice(len(dataset), subset_size, replace=False)
        subset = Subset(dataset, indices)

        train_loader = DataLoader(subset, batch_size=batch_size, shuffle=True, num_workers=4)

        print(f"\nTraining on {name} dataset with {factor * 100:.0f}% data")
        model = BeitClassifier(img_size=32, patch_size=8, num_classes=10).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=1e-4)

        test_accuracy = train_model(model, train_loader, test_loader, criterion, optimizer, device, epochs=5)
        results[name][factor] = test_accuracy

for factor in reduction_factors:
    balanced_acc = results["Balanced"][factor]
    for name in ["Exp Imbalanced", "Step Imbalanced"]:
        acc_diff = balanced_acc - results[name][factor]
        print(f"Accuracy Difference ({name} vs Balanced) at {factor * 100:.0f}% data: {acc_diff:.2f}%")


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Using device: cuda

Training on Balanced dataset with 25% data


config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

Epoch 1/5: 100%|██████████| 98/98 [00:04<00:00, 19.69it/s]


Epoch 1: Loss = 1.9005, Accuracy = 29.34%


Epoch 2/5: 100%|██████████| 98/98 [00:04<00:00, 20.91it/s]


...

Epoch 4: Loss = 0.8635, Accuracy = 67.91%


Epoch 5/5: 100%|██████████| 198/198 [00:09<00:00, 21.12it/s]


Epoch 5: Loss = 0.7744, Accuracy = 71.64%


Evaluating: 100%|██████████| 79/79 [00:01<00:00, 53.35it/s]

Accuracy Difference (Exp Imbalanced vs Balanced) at 25% data: 21.59%
Accuracy Difference (Step Imbalanced vs Balanced) at 25% data: 16.02%
Accuracy Difference (Exp Imbalanced vs Balanced) at 50% data: 25.60%
Accuracy Difference (Step Imbalanced vs Balanced) at 50% data: 19.77%
Accuracy Difference (Exp Imbalanced vs Balanced) at 75% data: 23.01%
Accuracy Difference (Step Imbalanced vs Balanced) at 75% data: 21.79%
Accuracy Difference (Exp Imbalanced vs Balanced) at 100% data: 26.57%
Accuracy Difference (Step Imbalanced vs Balanced) at 100% data: 23.68%


In [52]:
data = []
for factor in reduction_factors:
    row = {"Reduction Factor": f"{int(factor * 100)}%"}
    for name in ["Exp Imbalanced", "Step Imbalanced", "Balanced"]:
        row[name] = results[name][factor]
    data.append(row)
pd.DataFrame(data)

,Reduction Factor,Exp Imbalanced,Step Imbalanced,Balanced
0,25%,23.85,29.42,45.44
1,50%,25.94,31.77,51.54
2,75%,30.54,31.76,53.55
3,100%,29.51,32.40,56.08


# MAE

In [54]:
import torch
import torch.nn as nn
from torch.hub import load
from transformers import ViTMAEConfig, ViTMAEModel
from timm.models.layers import to_2tuple


class ViTMAEClassifier(nn.Module):
    def __init__(self, img_size=32, patch_size=8, num_classes=10):
        super(ViTMAEClassifier, self).__init__()
        self.config = ViTMAEConfig.from_pretrained("facebook/vit-mae-base")
        self.config.image_size = img_size
        self.config.patch_size = patch_size
        self.mae = ViTMAEModel(self.config)
        self.classifier = nn.Linear(self.config.hidden_size, num_classes)

    def forward(self, x):
        outputs = self.mae(pixel_values=x)
        pooled_output = outputs.last_hidden_state.mean(dim=1)
        logits = self.classifier(pooled_output)
        return logits


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
exp_dataset = create_imbalanced_dataset(datasets.CIFAR10(root='./data', train=True, download=True, transform=transform), 'exp', 0.01)
step_dataset = create_imbalanced_dataset(datasets.CIFAR10(root='./data', train=True, download=True, transform=transform), 'step', 0.01)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

batch_size = 128
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

reduction_factors = [0.25, 0.5, 0.75, 1]
results = {}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

for name, dataset in [("Balanced", train_dataset), ("Exp Imbalanced", exp_dataset), ("Step Imbalanced", step_dataset)]:
    results[name] = {}
    for factor in reduction_factors:
        subset_size = int(len(dataset) * factor)
        indices = np.random.choice(len(dataset), subset_size, replace=False)
        subset = Subset(dataset, indices)

        train_loader = DataLoader(subset, batch_size=batch_size, shuffle=True, num_workers=4)

        print(f"\nTraining on {name} dataset with {factor * 100:.0f}% data")
        model = ViTMAEClassifier(img_size=32, patch_size=8, num_classes=10).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=1e-4)

        test_accuracy = train_model(model, train_loader, test_loader, criterion, optimizer, device, epochs=5)
        results[name][factor] = test_accuracy

for factor in reduction_factors:
    balanced_acc = results["Balanced"][factor]
    for name in ["Exp Imbalanced", "Step Imbalanced"]:
        acc_diff = balanced_acc - results[name][factor]
        print(f"Accuracy Difference ({name} vs Balanced) at {factor * 100:.0f}% data: {acc_diff:.2f}%")


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Using device: cuda

Training on Balanced dataset with 25% data


config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

Epoch 1/5: 100%|██████████| 98/98 [00:02<00:00, 41.82it/s]


Epoch 1: Loss = 2.3265, Accuracy = 15.43%


Epoch 2/5: 100%|██████████| 98/98 [00:02<00:00, 42.21it/s]


...

Epoch 4: Loss = 1.2721, Accuracy = 49.50%


Epoch 5/5: 100%|██████████| 198/198 [00:04<00:00, 46.95it/s]


Epoch 5: Loss = 1.2393, Accuracy = 51.40%


Evaluating: 100%|██████████| 79/79 [00:00<00:00, 92.39it/s] 

Accuracy Difference (Exp Imbalanced vs Balanced) at 25% data: 11.09%
Accuracy Difference (Step Imbalanced vs Balanced) at 25% data: 6.16%
Accuracy Difference (Exp Imbalanced vs Balanced) at 50% data: 18.14%
Accuracy Difference (Step Imbalanced vs Balanced) at 50% data: 12.79%
Accuracy Difference (Exp Imbalanced vs Balanced) at 75% data: 17.10%
Accuracy Difference (Step Imbalanced vs Balanced) at 75% data: 10.49%
Accuracy Difference (Exp Imbalanced vs Balanced) at 100% data: 17.44%
Accuracy Difference (Step Imbalanced vs Balanced) at 100% data: 10.58%


In [55]:
data = []
for factor in reduction_factors:
    row = {"Reduction Factor": f"{int(factor * 100)}%"}
    for name in ["Exp Imbalanced", "Step Imbalanced", "Balanced"]:
        row[name] = results[name][factor]
    data.append(row)
pd.DataFrame(data)

,Reduction Factor,Exp Imbalanced,Step Imbalanced,Balanced
0,25%,17.05,21.98,28.14
1,50%,16.65,22.00,34.79
2,75%,18.73,25.34,35.83
3,100%,19.84,26.70,37.28
